In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=1000,
    return_messages=True,
    memory_key="history",
)


examples = [
    {
        "movie_title": "탑건",
        "answer": """
        "탑건" -> "🛩️👨‍✈️🔥",
        """,
    },
    {
        "movie_title": "대부",
        "answer": """
            "대부" -> "👨‍👨‍👦🔫🍝",
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "Make a imoji based on the movie titled {movie_title}"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Let's make some imojis based on the movie titles. if you know the movie I asked, just use 3 emojis." ),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "Ok, let's make some more imojis based on the {movie_title}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(result.content)

invoke_chain("나홀로집에")

invoke_chain("인셉션")

invoke_chain("아바타")

"나홀로집에" -> "🏠👦🎄"
"인셉션" -> "🧠💭🌌"
"아바타" -> "🌿🌎💧"


In [14]:
invoke_chain("이전에 어떤 영화를 물어봤었지?")

Sure! Here are the emojis based on the previously asked movie titles:

- "탑건" -> "🛩️👨‍✈️🔥"
- "대부" -> "👨‍👨‍👦🔫🍝"
- "나홀로집에" -> "🏠👦🎄"
- "인셉션" -> "🧠💭🌌"
- "아바타" -> "🌿🌎💧"
